In [ ]:
## 코드 설명 :
# (1) (월/분기/반기/연) 결과 통합
# (2) 최소 '연 MSE' 값 기준의 최적 모델 선택

------------
## 0. 환경설정

In [7]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import time
import glob
import pickle
import itertools

import warnings
warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션
pd.set_option('display.max_columns', 100)

#지수표현
pd.options.display.float_format = '{:.5f}'.format

---------
## 1. 입력값 기입

In [8]:
# data_folder : 원본데이터 위치(폴더명)
# save_folder : 변환데이터 저장 위치(폴더명)
# file_nm : 파일명
# y_colnm : y 컬럼명

data_folder = 'result'
save_folder = 'result/mody'
file_nm = ['result_month_merge_cls_new',
           'result_quarter_merge_cls_new',
           'result_half_merge_cls_new',
           'result_year_merge_cls_new']
y_colnm = ['SEP_CNT','SEP_CNT','SEP_CNT','SEP_CNT']

division_list = ['cls']  # age, dis 추가 가능(위의 file_nm 리스트에 맞는 구분자 입력)

In [9]:
# 파일 정보 통합
file_info = pd.DataFrame(file_nm).rename(columns = {0:'file_nm'})
file_info['y_colnm'] = y_colnm

---------
## 1. 각 결과별 '연 MSE' 값 계산

In [10]:
for divis in division_list:
    divis_file_info = file_info.loc[file_info['file_nm'].str.contains('cls')].reset_index(drop=True)
    
    nujuk_yr_result_val = pd.DataFrame()  # 전체 결과값 확인
    nujuk_yr_result = pd.DataFrame()      # MSE값 비교

    for file_num in range(len(divis_file_info)):

        file_nm = divis_file_info['file_nm'][file_num]
        y_colnm = divis_file_info['y_colnm'][file_num]

        data = pd.read_csv(data_folder + '/' + file_nm + '.csv', encoding = 'utf-8')

        data['stand_time'] = data['stand_time'].astype('str')     # 컬럼 타입 변환
        data['YEAR'] = data['stand_time'].map(lambda x : x[0:4])  # YEAR 컬럼 생성

        # target별로 YEAR 컬럼으로 grouping하여 연값 생성
        gb_yr = data.groupby(['target','YEAR']).sum().reset_index()
        gb_yr['YR_MSE'] = gb_yr['DIFF'] * gb_yr['DIFF']                               # YR_DIFF가 연값(1개)이므로 MSE 계산 시 1로 나눠줌(여기서는 1 나누기 생략)        
        gb_yr['CLASS'] = file_nm                                                      # 파일명 구분 (월/분기/반기/연)
        gb_yr = pd.merge(gb_yr, data[['target','YEAR','BEST_MDL']], how = 'left')
        gb_yr = gb_yr.sort_values(by = ['CLASS','target','YEAR'])                   # target명 순으로 정렬
        gb_yr['CLASS'] = gb_yr['CLASS'].map(lambda x : x.split('_')[1])

        nujuk_yr_result_val = nujuk_yr_result_val.append(gb_yr)

        tmp = gb_yr[['CLASS','target','YEAR','YR_MSE']]
        tmp.rename(columns = {'YR_MSE':list(gb_yr['CLASS'].unique())[0]}, inplace=True)
        tmp = tmp[['target','YEAR',list(gb_yr['CLASS'].unique())[0]]]

        if file_num == 0:
            nujuk_yr_result = tmp
        else:
            nujuk_yr_result = pd.merge(nujuk_yr_result, tmp, how = 'left')

    nujuk_yr_result['MIN_MSE'] = np.NaN
    nujuk_yr_result['CLASS'] = ''

    for num in range(len(nujuk_yr_result)):
        nujuk_yr_result['MIN_MSE'][num] = nujuk_yr_result.iloc[num][['month','quarter','half','year']].min()

        if nujuk_yr_result['MIN_MSE'][num] == nujuk_yr_result['month'][num]:
            nujuk_yr_result['CLASS'][num] = 'month'
        elif nujuk_yr_result['MIN_MSE'][num] == nujuk_yr_result['quarter'][num]:
            nujuk_yr_result['CLASS'][num] = 'quarter'
        elif nujuk_yr_result['MIN_MSE'][num] == nujuk_yr_result['half'][num]:
            nujuk_yr_result['CLASS'][num] = 'half'
        elif nujuk_yr_result['MIN_MSE'][num] == nujuk_yr_result['year'][num]:
            nujuk_yr_result['CLASS'][num] = 'year'
        else :
            pass

    nujuk_yr_result_val = nujuk_yr_result_val.drop_duplicates().reset_index(drop=True)
    nujuk_yr_result = nujuk_yr_result.drop_duplicates().reset_index(drop=True)

    tot_result = pd.merge(nujuk_yr_result[['target','YEAR','CLASS']], nujuk_yr_result_val, on = ['CLASS','target','YEAR'], how = 'left')
    tot_result.rename(columns = {'CLASS':'SELECTED_MODEL','target':'TARGET'}, inplace=True)
    
    tot_result.to_csv(save_folder + '/Total_Result_' + divis + '.csv', index=False)

-------------